In [ ]:
# -*- coding: utf-8 -*-
"""train_moco.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/你的笔记本ID

# 🚀 MoCo模型训练笔记本
## 符武文使用

这个笔记本用于训练MoCo（Momentum Contrast）对比学习模型。
"""

# [1] 安装依赖
print("步骤1: 安装依赖包...")
!pip install torch torchvision matplotlib scikit-learn pandas seaborn tqdm pyyaml -q

print("✅ 依赖安装完成")

# [2] 检查GPU环境
print("\n步骤2: 检查GPU环境...")
import torch
import sys

print(f"Python版本: {sys.version}")
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    device_props = torch.cuda.get_device_properties(0)
    print(f"GPU型号: {device_name}")
    print(f"GPU内存: {device_props.total_memory / 1024**3:.1f} GB")
    print(f"CUDA能力: {device_props.major}.{device_props.minor}")
else:
    print("⚠️  警告: 未检测到GPU，将使用CPU训练（速度会很慢）")

# [3] 挂载Google Drive（可选，用于保存结果）
print("\n步骤3: 挂载Google Drive（可选）...")
from google.colab import drive

try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive已挂载")
    use_drive = True
except:
    print("⚠️  Google Drive挂载失败，结果将保存在Colab临时空间")
    use_drive = False

# [4] 下载项目代码
print("\n步骤4: 下载项目代码...")
import os
import shutil

# 清理旧文件
if os.path.exists('/content/contrastive_simple'):
    print("清理旧文件...")
    shutil.rmtree('/content/contrastive_simple')

# 从GitHub下载代码
print("从GitHub下载代码...")
!git clone https://github.com/your-username/contrastive_simple.git

# 进入项目目录
%cd contrastive_simple

print("\n✅ 项目结构:")
!ls -la

# [5] 查看配置文件
print("\n步骤5: 查看MoCo配置文件...")
!cat configs/moco.yaml

# [6] 训练MoCo模型
print("\n" + "="*60)
print("🚀 步骤6: 开始训练MoCo模型")
print("="*60)

import time
import subprocess

# 记录开始时间
start_time = time.time()

# 运行训练命令
print("开始训练MoCo模型...")
print("这可能需要2-4小时，具体取决于GPU性能")

# 使用subprocess以便实时查看输出
command = "python train.py --model moco --output /content/results --config configs/moco.yaml"
process = subprocess.Popen(
    command,
    shell=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# 实时输出训练日志
for line in process.stdout:
    print(line, end='')

# 等待训练完成
return_code = process.wait()

# 记录结束时间
end_time = time.time()
training_time = end_time - start_time

print(f"\n⏱️  训练总时间: {training_time/3600:.2f} 小时")

if return_code == 0:
    print("✅ MoCo训练成功完成！")
else:
    print(f"❌ 训练失败，返回码: {return_code}")

# [7] 查看训练结果
print("\n步骤7: 查看训练结果...")

print("📁 结果目录结构:")
!find /content/results/moco -type f -name "*.pt" -o -name "*.json" -o -name "*.npy" | sort

# 查看训练历史
import json
import pandas as pd

history_path = '/content/results/moco/training_history.json'
if os.path.exists(history_path):
    print("\n📊 训练统计:")
    with open(history_path, 'r') as f:
        history = json.load(f)
    
    # 创建DataFrame便于查看
    df_history = pd.DataFrame({
        'epoch': range(1, len(history.get('train_loss', [])) + 1),
        'loss': history.get('train_loss', []),
        'accuracy': history.get('train_acc', []) if 'train_acc' in history else [0] * len(history.get('train_loss', []))
    })
    
    print(f"总轮次: {len(df_history)}")
    print(f"最佳准确率: {history.get('best_acc', 0):.2%}")
    print(f"总训练时间: {history.get('total_time', 0)/3600:.2f} 小时")
    
    # 显示最后5个epoch的结果
    print("\n最后5个epoch的结果:")
    print(df_history.tail())
    
    # 绘制训练曲线
    if len(df_history) > 0:
        print("\n📈 绘制训练曲线...")
        import matplotlib.pyplot as plt
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
        
        # 损失曲线
        ax1.plot(df_history['epoch'], df_history['loss'], 'b-', linewidth=2)
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.set_title('MoCo训练损失曲线')
        ax1.grid(True, alpha=0.3)
        
        # 准确率曲线
        ax2.plot(df_history['epoch'], df_history['accuracy'], 'r-', linewidth=2)
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy')
        ax2.set_title('MoCo训练准确率曲线')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
else:
    print("⚠️  未找到训练历史文件")

# [8] 运行线性评估（可选）
print("\n步骤8: 运行线性评估（可选）...")

run_eval = input("是否运行线性评估？(y/n): ").lower().strip()

if run_eval == 'y':
    print("运行线性评估...")
    !python evaluate.py --model moco --input /content/results/moco --output /content/results/evaluation
    
    # 查看评估结果
    eval_result_path = '/content/results/evaluation/moco_linear_eval.json'
    if os.path.exists(eval_result_path):
        with open(eval_result_path, 'r') as f:
            eval_results = json.load(f)
        
        print("\n📈 线性评估结果:")
        print(f"  模型: {eval_results.get('model', 'moco')}")
        print(f"  训练准确率: {eval_results.get('linear_train_accuracy', 0):.2%}")
        print(f"  测试准确率: {eval_results.get('linear_test_accuracy', 0):.2%}")
        print(f"  训练样本数: {eval_results.get('num_train_samples', 0)}")
        print(f"  测试样本数: {eval_results.get('num_test_samples', 0)}")
else:
    print("跳过线性评估")

# [9] 保存结果到Google Drive
if use_drive:
    print("\n步骤9: 保存结果到Google Drive...")
    
    # 创建保存目录
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    drive_save_dir = f'/content/drive/MyDrive/contrastive_results/moco_{timestamp}'
    
    !mkdir -p {drive_save_dir}
    
    # 复制结果文件
    !cp -r /content/results/moco/* {drive_save_dir}/
    
    print(f"✅ 结果已保存到Google Drive: {drive_save_dir}")
    
    # 列出保存的文件
    print("\n📁 Google Drive中的文件:")
    !find {drive_save_dir} -type f | head -10

# [10] 下载结果到本地
print("\n步骤10: 下载结果到本地计算机...")
from google.colab import files

# 创建压缩包
print("创建压缩包...")
!zip -r moco_results.zip /content/results/moco/

# 检查文件大小
import os
zip_size = os.path.getsize('moco_results.zip') / 1024**2  # MB
print(f"压缩包大小: {zip_size:.1f} MB")

if zip_size > 100:
    print("⚠️  文件较大，下载可能需要一些时间")

# 下载
print("开始下载...")
try:
    files.download('moco_results.zip')
    print("✅ 下载完成！")
except Exception as e:
    print(f"❌ 下载失败: {e}")
    print("请尝试手动下载: 在左侧文件浏览器中右键点击 'moco_results.zip' -> 下载")

# [11] 训练总结
print("\n" + "="*60)
print("✨ MoCo训练完成总结")
print("="*60)
print(f"训练者: 符武文")
print(f"训练时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"训练时长: {training_time/3600:.2f} 小时")

if os.path.exists(history_path):
    with open(history_path, 'r') as f:
        history = json.load(f)
    print(f"最佳准确率: {history.get('best_acc', 0):.2%}")
    print(f"总训练轮次: {len(history.get('train_loss', []))}")

print("\n📁 生成的文件:")
!find /content/results/moco -type f -name "*" | while read file; do
    size=$(stat -c%s "$file" 2>/dev/null || stat -f%z "$file")
    size_mb=$(echo "scale=2; $size/1024/1024" | bc)
    echo "  $(basename "$file") (${size_mb}MB)"
done

print("\n🔗 分享步骤:")
print("1. 将下载的 'moco_results.zip' 文件")
print("2. 上传到微信群或共享网盘")
print("3. 通知刘长蒙和纪占锋进行分析")
print("="*60)

# [12] 清理临时文件（可选）
print("\n步骤12: 清理临时文件（可选）...")

cleanup = input("是否清理临时文件以释放空间？(y/n): ").lower().strip()

if cleanup == 'y':
    print("清理中...")
    !rm -rf /content/contrastive_simple
    !rm -rf /content/results
    !rm -f moco_results.zip
    print("✅ 临时文件已清理")
else:
    print("保留临时文件")

print("\n🎉 MoCo训练任务全部完成！")
print("感谢符武文的辛勤工作！")